In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob

In [2]:
raw_dat_dir = '/home/grace/Documents/python/acute_oral/data/raw/'
processed_dat_dir = '/home/grace/Documents/python/acute_oral/data/processed/'
interim_dat_dir = '/home/grace/Documents/python/acute_oral/data/interim/'
figures_dir = '/home/grace/Documents/python/acute_oral/reports/figures/'
test_dir = '/home/grace/Documents/python/acute_oral/data/external/TEST/'
times_dir = '/home/grace/Documents/python/acute_oral/data/external/TIMES/'
interim_dir = '/home/grace/Documents/python/acute_oral/data/interim/'

all overlap data 

In [3]:
relevant_acute_pred =pd.read_csv(interim_dir+'relevant_acute_pred.csv')

In [5]:
mask2 = (~relevant_acute_pred['known_pLD50_minus_TEST'].isnull()) & (~relevant_acute_pred['known_pLD50_minus_TIMES'].isnull())
tt_overlap = relevant_acute_pred[mask2]


In [7]:
tt_overlap.shape

(274, 21)

In [40]:
tt_overlap[tt_overlap['CAS'] == '97-86-9']

,Unnamed: 0,CAS,known_LD50_mgkg,TEST_pLD50,chem_name,predicted_toxicological_category,known_LD50_TIMES,TIMES_LD50_mgkg,found_by,DTXSID,...,average_mass,qsar_ready_smiles,smiles,known_pLD50,TIMES_pLD50,TEST_LD50_mgkg,known_pLD50_minus_TEST,known_pLD50_minus_TIMES,TIMES,TEST
8840,8840,97-86-9,9590.0,1.61,DTXSID3025461,Methacrylic acid esters,no data,5280.0,CAS-RN,DTXSID3025461,...,142.198,CC(C)COC(=O)C(C)=C,CC(C)COC(=O)C(C)=C,1.171075,1.43026,3490.546984,-0.438925,-0.259185,Below_CI,Below_CI


In [9]:
tt_overlap.columns

Index(['Unnamed: 0', 'CAS', 'known_LD50_mgkg', 'TEST_pLD50', 'chem_name',
       'predicted_toxicological_category', 'known_LD50_TIMES',
       'TIMES_LD50_mgkg', 'found_by', 'DTXSID', 'name', 'average_mass',
       'qsar_ready_smiles', 'smiles', 'known_pLD50', 'TIMES_pLD50',
       'TEST_LD50_mgkg', 'known_pLD50_minus_TEST', 'known_pLD50_minus_TIMES',
       'TIMES', 'TEST'],
      dtype='object')

In [15]:
ids = tt_overlap.CAS.unique().tolist()

TxP fingerprints

In [10]:
txps = pd.read_csv(raw_dat_dir+'Acute_SMILES_Toxprints.tsv', sep = '\t')

In [17]:
tt_txps = txps[txps['M_NAME'].isin(ids)]

In [21]:
txps_df = pd.melt(tt_txps, id_vars = ['M_NAME'], value_vars = tt_txps.columns.tolist()[1:])

In [25]:
txps_df = txps_df.replace(0,np.nan)
    

In [31]:
txps_df = txps_df.dropna(how = 'any')

In [35]:
txps_df = txps_df.sort_values(by = 'M_NAME', ascending = True)

Txps for specific chemical 97-86-9

In [45]:
txp_chem = txps_df[txps_df['M_NAME'] == '97-86-9']['variable'].tolist()

In [47]:
txp_chem

['bond:C(=O)O_carboxylicEster_alkenyl',
 'bond:C=O_carbonyl_generic',
 'chain:alkeneLinear_mono-ene_ethylene_generic',
 'chain:alkeneBranch_mono-ene_2-butene',
 'bond:C(=O)O_carboxylicEster_acyclic',
 'chain:alkaneLinear_ethyl_C2_(connect_noZ_CN=4)',
 'chain:alkaneBranch_isopropyl_C3']

Load probabilities based on TxPs

In [43]:
pivot_probs= pd.read_csv(interim_dat_dir+'all_pivot_probs.csv')

In [48]:
example_prob = pivot_probs[pivot_probs['TxP'].isin(txp_chem)]


In [60]:
example_prob['TEST']

18     0.049467
20     0.029098
46     0.477207
245    0.138700
257    0.286130
265    0.011639
279    0.064985
Name: TEST, dtype: float64

In [61]:
example_prob['TIMES']

18     0.048077
20     0.057692
46     0.387821
245    0.083333
257    0.221154
265    0.025641
279    0.131410
Name: TIMES, dtype: float64

Probability of all TxPs in TEST being outside of domain

In [58]:
0.049467*0.029098*0.477207*0.138700*0.286130*0.011639*0.064985

2.0618362232774318e-08

In [69]:
np.prod(example_prob['TEST'])

2.0618638236171015e-08

Probability of all TxPs in TIMES being outside of domain

In [62]:
0.048077*0.057692*0.387821*0.083333*0.221154*0.025641*0.131410

6.679738580684652e-08

In [70]:
np.prod(example_prob['TIMES'])

6.679796913287913e-08

Is the fact that prob for TIMES > TEST related to the fact that the prediction from TIMES should be treated with higher confidence even though both are out of the domain???

In [63]:
txp_chem2 = txps_df[txps_df['M_NAME'] == '3734-33-6']['variable'].tolist()

In [64]:
txp_chem2 

['bond:C=O_carbonyl_generic',
 'bond:quatN_generic',
 'chain:aromaticAlkane_Ph-C1_acyclic_connect_noDblBd',
 'chain:aromaticAlkane_Ph-C1_acyclic_connect_H_gt_1',
 'group:ligand_path_4_bidentate_aminoacetaldehyde',
 'bond:quatN_alkyl_acyclic',
 'bond:quatN_b-carbonyl',
 'chain:aromaticAlkane_Ar-C_meta',
 'group:aminoAcid_aminoAcid_generic',
 'ring:aromatic_benzene',
 'chain:aromaticAlkane_Ph-C1_acyclic_generic',
 'bond:C(=O)N_carboxamide_generic',
 'bond:NC=O_aminocarbonyl_generic',
 'bond:C(=O)N_carboxamide_(NHR)',
 'chain:alkaneLinear_ethyl_C2(H_gt_1)',
 'ring:aromatic_phenyl']

In [65]:
example_prob2 = pivot_probs[pivot_probs['TxP'].isin(txp_chem2)]

In [71]:
print(np.prod(example_prob2['TEST']))
print(np.prod(example_prob2['TIMES']))

4.1483195799035443e-16
7.929966413210253e-16


In [103]:
enrich_pivot = pd.read_csv(interim_dat_dir+'enrich_pivot.csv', header = [0,1])

In [105]:
enrich_pivot.columns.droplevel(1)

Index(['Unnamed: 0_level_0', 'OR', 'OR', 'p', 'p'], dtype='object')

In [106]:
enrich_pivot.columns = enrich_pivot.columns.droplevel(1)

In [107]:
enrich_pivot = enrich_pivot.iloc[1:,:]

In [109]:
cols=pd.Series(enrich_pivot.columns)

for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
enrich_pivot.columns=cols

In [110]:
enrich_pivot

,Unnamed: 0_level_0,OR,OR.1,p,p.1
1,atom:element_metal_metalloid,1.026810,2.467532,1.000000,0.654153
2,bond:C#N_cyano_acylcyanide,inf,inf,1.000000,1.000000
3,bond:C#N_nitrile,0.969034,inf,1.000000,0.087484
4,bond:C#N_nitrile_generic,0.785937,inf,0.572558,0.087484
5,bond:C(=O)N_carbamate,1.402709,0.242942,0.454799,0.008783
...,...,...,...,...,...
309,ring:hetero_[7]_generic_1-Z,1.474827,0.000000,0.530059,0.379722
310,ring:hetero_[7]_generic_1_4-Z,0.853346,0.292484,0.681030,0.013476
311,ring:polycycle_bicyclo_[2.1.1]heptane,1.427875,0.000000,1.000000,0.379722
312,ring:polycycle_bicyclo_[3.3.1]nonane,0.569903,0.000000,1.000000,0.379722


In [113]:
chem_2 = enrich_pivot[enrich_pivot['Unnamed: 0_level_0'].isin(txp_chem2)]

TEST OR prod

In [115]:
np.prod(chem_2['OR'])

0.9206251159436315

TIMES OR prod

In [116]:
np.prod(chem_2['OR.1'])

9.069585219411078